In [1]:
import pandas as pd

## Multi-index

In [5]:
data = {
    'Year' : [2019, 2019, 2020, 2020, 2021, 2021],
    'Quarter' : [1, 2, 1, 2, 1, 2],
    'Revenue' : [100, 120, 110, 130, 105, 125]
}

df = pd.DataFrame(data)
df_multi = df.set_index(['Year', 'Quarter'])
display(df_multi)

Revenue
Year Quarter         
2019 1            100
     2            120
2020 1            110
     2            130
2021 1            105
     2            125

In [7]:
display(df_multi.loc[2020])
display(df_multi.loc[2020].loc[1].loc['Revenue'])
idxs = (df_multi.index
                .get_level_values('Quarter') == 1)
display(idxs)
display(df_multi[idxs])

,Revenue
Quarter,
1,110
2,130


110

array([ True, False,  True, False,  True, False])

,,Revenue
Year,Quarter,
2019,1,100
2020,1,110
2021,1,105


In [13]:
df_multi

Revenue
Year Quarter         
2019 1            100
     2            120
2020 1            110
     2            130
2021 1            105
     2            125